In [1]:
import os

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
from glob import glob
import os
import json 
from sklearn.metrics import f1_score

In [3]:
path = os.getcwd()

In [4]:
# 분석에 사용할 feature 선택
csv_features = ['내부 온도 1 평균', '내부 온도 1 최고', '내부 온도 1 최저', '내부 습도 1 평균', '내부 습도 1 최고', 
                '내부 습도 1 최저', '내부 이슬점 평균', '내부 이슬점 최고', '내부 이슬점 최저']

# csv_files = sorted(glob('sample_data/*/*.csv'))
csv_files = sorted(glob(path + '\\sample_data\\sample_data\\*\\*.csv'))

temp_csv = pd.read_csv(csv_files[0])[csv_features]
max_arr, min_arr = temp_csv.max().to_numpy(), temp_csv.min().to_numpy()

# feature 별 최대값, 최솟값 계산
for csv in tqdm(csv_files[1:]):
    temp_csv = pd.read_csv(csv)[csv_features]
    temp_max, temp_min = temp_csv.max().to_numpy(), temp_csv.min().to_numpy()
    max_arr = np.max([max_arr,temp_max], axis=0)
    min_arr = np.min([min_arr,temp_min], axis=0)

# feature 별 최대값, 최솟값 dictionary 생성
csv_feature_dict = {csv_features[i]:[min_arr[i], max_arr[i]] for i in range(len(csv_features))}
csv_feature_dict

100%|██████████| 498/498 [00:01<00:00, 252.65it/s]


{'내부 온도 1 평균': [14.4, 47.3],
 '내부 온도 1 최고': [14.5, 47.6],
 '내부 온도 1 최저': [14.4, 47.0],
 '내부 습도 1 평균': [34.1, 100.0],
 '내부 습도 1 최고': [36.5, 100.0],
 '내부 습도 1 최저': [32.4, 100.0],
 '내부 이슬점 평균': [12.4, 29.9],
 '내부 이슬점 최고': [12.8, 31.9],
 '내부 이슬점 최저': [12.1, 29.1]}

In [5]:
# 제공된 sample data는 파프리카와 시설포도 2종류의 작물만 존재
label_description = {
 '3_00_0': '파프리카_정상',
 '3_a9_1': '파프리카흰가루병_초기',
 '3_a9_2': '파프리카흰가루병_중기',
 '3_a9_3': '파프리카흰가루병_말기',
 '3_a10_1': '파프리카잘록병_초기',
 '3_a10_2': '파프리카잘록병_중기',
 '3_a10_3': '파프리카잘록병_말기',
 '3_b3_1': '칼슘결핍_초기',
 '3_b3_2': '칼슘결핍_중기',
 '3_b3_3': '칼슘결핍_말기',
 '3_b6_1': '다량원소결핍 (N)_초기',
 '3_b6_2': '다량원소결핍 (N)_중기',
 '3_b6_3': '다량원소결핍 (N)_말기',
 '3_b7_1': '다량원소결핍 (P)_초기',
 '3_b7_2': '다량원소결핍 (P)_중기',
 '3_b7_3': '다량원소결핍 (P)_말기',
 '3_b8_1': '다량원소결핍 (K)_초기',
 '3_b8_2': '다량원소결핍 (K)_중기',
 '3_b8_3': '다량원소결핍 (K)_말기',
 '6_00_0': '시설포도_정상',
 '6_a11_1': '시설포도탄저병_초기',
 '6_a11_2': '시설포도탄저병_중기',
 '6_a11_3': '시설포도탄저병_말기',
 '6_a12_1': '시설포도노균병_초기',
 '6_a12_2': '시설포도노균병_중기',
 '6_a12_3': '시설포도노균병_말기',
 '6_b4_1': '일소피해_초기',
 '6_b4_2': '일소피해_중기',
 '6_b4_3': '일소피해_말기',
 '6_b5_1': '축과병_초기',
 '6_b5_2': '축과병_중기',
 '6_b5_3': '축과병_말기',
}

label_encoder = {key:idx for idx, key in enumerate(label_description)}
label_decoder = {val:key for key, val in label_encoder.items()}

In [6]:
csv_features

['내부 온도 1 평균',
 '내부 온도 1 최고',
 '내부 온도 1 최저',
 '내부 습도 1 평균',
 '내부 습도 1 최고',
 '내부 습도 1 최저',
 '내부 이슬점 평균',
 '내부 이슬점 최고',
 '내부 이슬점 최저']

In [7]:
data_files = glob(path + '\\sample_data\\sample_data\\*')
len(csv_feature_dict.keys())



# csvset = np.empty((-1,len(csv_feature_dict.keys())), float)
# imageset = []
# jsonset = []

# len(df)
# np.empty((0,10,11), float)

# #시계열데이터 길이 다른것 많음
# for i in data_files:
#     csv_path = glob(data_files[i]+"\\*.csv")[0]
#     json_path = glob(data_files[i]+"\\*.json")
#     image_path = glob(data_files[i]+"\\*.jpg")
    
#     df = pd.read_csv(csv_path)
#     df = df[csv_features]
    
#     # MinMax scaling
#     for col in csv_feature_dict.keys():
#         df[col] = df[col] - csv_feature_dict[col][0]
#         df[col] = df[col] / (csv_feature_dict[col][1]-csv_feature_dict[col][0])
#     df = df.to_numpy() 
    
    
#     img = cv2.imread(image_path)
#     img = cv2.resize(img, dsize=(256, 256), interpolation=cv2.INTER_AREA)
#     img = img.astype(np.float32)/255
#     img = np.transpose(img, (2,0,1))
        
        
#     with open(json_path, 'r') as f:
#         json_file = json.load(f)
        
#         crop = json_file['annotations']['crop']
#         disease = json_file['annotations']['disease']
#         risk = json_file['annotations']['risk']
#         label = label_encoder[f'{crop}_{disease}_{risk}']
                    

9

In [9]:
data_files

['d:\\OneDrive - UOS\\allrepos\\dacon_diagnosis\\sample\\sample_data\\sample_data\\10000',
 'd:\\OneDrive - UOS\\allrepos\\dacon_diagnosis\\sample\\sample_data\\sample_data\\10001',
 'd:\\OneDrive - UOS\\allrepos\\dacon_diagnosis\\sample\\sample_data\\sample_data\\10002',
 'd:\\OneDrive - UOS\\allrepos\\dacon_diagnosis\\sample\\sample_data\\sample_data\\10003',
 'd:\\OneDrive - UOS\\allrepos\\dacon_diagnosis\\sample\\sample_data\\sample_data\\10004',
 'd:\\OneDrive - UOS\\allrepos\\dacon_diagnosis\\sample\\sample_data\\sample_data\\10005',
 'd:\\OneDrive - UOS\\allrepos\\dacon_diagnosis\\sample\\sample_data\\sample_data\\10006',
 'd:\\OneDrive - UOS\\allrepos\\dacon_diagnosis\\sample\\sample_data\\sample_data\\10007',
 'd:\\OneDrive - UOS\\allrepos\\dacon_diagnosis\\sample\\sample_data\\sample_data\\10008',
 'd:\\OneDrive - UOS\\allrepos\\dacon_diagnosis\\sample\\sample_data\\sample_data\\10009',
 'd:\\OneDrive - UOS\\allrepos\\dacon_diagnosis\\sample\\sample_data\\sample_data\\10010',

In [11]:
data_files = glob(path + '\\sample_data\\sample_data\\*')

class ControllerCSV():
    
    def road_csv(self,foldnam):
        df = pd.read_csv(path)
        return df
    
    def scaling(self,minmaxdic,df):
        for col in minmaxdic.keys():
            df[col] = df[col] - csv_feature_dict[col][0]
            df[col] = df[col] / (csv_feature_dict[col][1]-csv_feature_dict[col][0])
        return df
    
    def featureselection(self,featnam,df):
        df = df[featnam]
        return df
    
    def datafortensor(self,df,timelag):
        nparr = df.to_numpy() 
        collen = len(df.columns)
        
        predictor = np.empty((0,timelag,collen), float)
        for i in range(len(df)-timelag+1):
            if i+timelag > len(df):
                pass
            else:
                predictor = np.append(predictor,nparr[i:i+timelag,:].reshape(-1,timelag,collen),axis = 0)
        return predictor
        
csvcon = ControllerCSV()
# df = csvcon.road_csv('d:\\OneDrive - UOS\\allrepos\\dacon_diagnosis\\sample\\sample_data\\sample_data\\10000\\10000.csv')
df = csvcon.road_csv(path + '\\sample_data\\sample_data\\10000\\10000.csv')
df2 = csvcon.featureselection(csv_features,df)
# df3 = csvcon.scaling(csv_feature_dict,df2)
tensordata = csvcon.datafortensor(df2,10)
print(tensordata.shape,tensordata)
            
            

PermissionError: [Errno 13] Permission denied: 'd:\\OneDrive - UOS\\allrepos\\dacon_diagnosis\\sample'

..


In [19]:
glob(path + '\\sample_data\\sample_data\\10000\\*')

['d:\\OneDrive - UOS\\allrepos\\dacon_diagnosis\\sample\\sample_data\\sample_data\\10000\\10000.csv',
 'd:\\OneDrive - UOS\\allrepos\\dacon_diagnosis\\sample\\sample_data\\sample_data\\10000\\10000.jpg',
 'd:\\OneDrive - UOS\\allrepos\\dacon_diagnosis\\sample\\sample_data\\sample_data\\10000\\10000.json']

In [66]:
# time_lag = 10
# data = datamerge["KRW-ETH"]
# predictor = np.empty((0,10,11), float)
# target = np.empty((0,1), float)
# for i in range(len(data[0])-time_lag+1):
#     if i+time_lag+1 >= len(data[0]):
#         pass
#     else:
#         target = np.append(target,data[1][i+time_lag+1].reshape(-1,1),axis = 0)
#         predictor = np.append(predictor,data[0][i:i+time_lag,:].reshape(-1,10,11),axis = 0)

# print(predictor.shape)

9

In [26]:
import tensorflow as tf
from tensorflow import keras

In [27]:
class ComBlock(keras.layers.Layer):
    def __init__(self):
        super(ComBlock, self).__init__()
        self.layer1 = tf.keras.layers.LSTM(128)
        self.layer2 = tf.keras.layers.Dense(64)
        self.layer3 = tf.keras.layers.Dense(32)
        self.layer4 = tf.keras.layers.Dense(16)
        self.layer5 = tf.keras.layers.Dense(1)
        
    def call(self, inputs):
        x = self.layer1(inputs)
        x = tf.nn.relu(x)
        x = self.layer2(x)
        x = tf.nn.relu(x)
        x = self.layer3(x)
        x = tf.nn.relu(x)
        x = self.layer4(x)
        x = tf.nn.relu(x)
        x = self.layer5(x)
        return x
        

In [28]:
class Paramodel(keras.Model):
    def __init__(self, name = None):
        super(Paramodel, self).__init__()
    
        self.module1 = ComBlock()
        self.module2 = ComBlock()
        self.module3 = ComBlock()
        # self.outlayer1 = tf.keras.layers.Dense(16)
        self.outlayer = tf.keras.layers.Dense(1)
        
    def call(self, inputs):
        x1 = self.module1(inputs[0])
        x2 = self.module2(inputs[1])
        x3 = self.module3(inputs[2])
        
        x = tf.concat([x1,x2,x3],axis=1)
        
        
        return(self.outlayer(x))
    

In [29]:
inputs = tf.random.normal([3,2,10,8])
model1 = Paramodel(name='model_01')
print(model1(inputs))

tf.Tensor(
[[0.00146205]
 [0.00196941]], shape=(2, 1), dtype=float32)


In [30]:
model1.variables,model1.summary()

Model: "paramodel"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 com_block (ComBlock)        multiple                  81025     
                                                                 
 com_block_1 (ComBlock)      multiple                  81025     
                                                                 
 com_block_2 (ComBlock)      multiple                  81025     
                                                                 
 dense_12 (Dense)            multiple                  4         
                                                                 
Total params: 243,079
Trainable params: 243,079
Non-trainable params: 0
_________________________________________________________________


([<tf.Variable 'paramodel/com_block/lstm/lstm_cell/kernel:0' shape=(8, 512) dtype=float32, numpy=
  array([[-0.0245102 , -0.00012096,  0.08350237, ..., -0.09274025,
          -0.0441144 , -0.06993738],
         [ 0.00053825,  0.09411462,  0.05307796, ..., -0.05366554,
           0.05855351,  0.09965653],
         [-0.08741946,  0.09360541,  0.08218049, ..., -0.00605962,
          -0.0524269 , -0.08006962],
         ...,
         [-0.01174441, -0.02716009, -0.06097151, ...,  0.03948783,
           0.04528192,  0.01763778],
         [-0.07248637, -0.06298581, -0.02951048, ..., -0.05404836,
          -0.07776905,  0.00996447],
         [ 0.04051698, -0.00744795,  0.10221062, ...,  0.02306407,
          -0.0173823 , -0.03307955]], dtype=float32)>,
  <tf.Variable 'paramodel/com_block/lstm/lstm_cell/recurrent_kernel:0' shape=(128, 512) dtype=float32, numpy=
  array([[ 0.11958289, -0.04309463, -0.02117723, ...,  0.00200937,
           0.07356272,  0.00599375],
         [ 0.06400762,  0.037921